<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 4.1: Introduction to FIRRTL

**Prev: [Generators: Types](3.6_types.ipynb)**<br>
**Next: [FIRRTL AST Traversal](4.2_firrtl_ast_traversal.ipynb)**

## Motivation
你已经学习了一些 Scala 并编写了一些 Chisel，对于 90% 的用户来说，这应该足以成为 Chisel 的爱好者。

然而，一些用例更适合作为 Chisel 设计的程序化转换，而不是作为生成器。

例如，假设我们想计算设计中的寄存器数量。这作为生成器来做会很困难，因此我们可以编写一个 FIRRTL pass 来为我们完成它。

## Setup

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}
import firrtl._

## What is FIRRTL?
你可能已经注意到，当你执行一个 Chisel 设计时，它会 elaborate（执行周围的 Scala 代码）来构建你的生成器实例，并解决所有 Scala 参数。

Chisel 并不是直接生成 Verilog，而是生成一个称为 FIRRTL 的中间表示，它表示 elaborate 后的（参数已解决的）RTL 实例。它可以序列化（转换为字符串以写入文件），并且这种序列化的语法是人类可读的。然而，在内部，它并不是表示为一个长字符串，而是作为一个节点树组织的数据结构，称为抽象语法树（AST）。

让我们看看！我们将一个简单的 Chisel 设计 elaborate，并检查它生成的 FIRRTL！

首先，我们定义一个 Chisel 模块，它将输入信号延迟两个周期。

In [ ]:
class DelayBy2(width: Int) extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(width.W))
    val out = Output(UInt(width.W))
  })
  val r0 = RegNext(io.in)
  val r1 = RegNext(r0)
  io.out := r1
}

Next, let's elaborate it, serialize, and print out the FIRRTL it generates.

In [ ]:
println(chisel3.Driver.emit(() => new DelayBy2(4)))

正如你所看到的，序列化的 FIRRTL 看起来与我们的 Chisel 设计非常相似，所有生成器参数都已解决。

## The FIRRTL AST

如前所述，FIRRTL 表示可以序列化为字符串，但在内部，它是一个称为 AST（抽象语法树）的数据结构。该数据结构是一个节点树，其中一个节点可以包含子节点。这个数据结构中没有循环。

让我们看看内部数据结构的样子：

In [ ]:
val firrtlSerialization = chisel3.Driver.emit(() => new DelayBy2(4))
val firrtlAST = firrtl.Parser.parse(firrtlSerialization.split("\n").toIterator, Parser.GenInfo("file.fir"))

println(firrtlAST)

显然，数据结构的序列化看起来并不那么美观，但你可以看到一些类和内部表示 RTL 设计的内容。让我们尝试美化一下，使其更易于理解。

In [ ]:
println(stringifyAST(firrtlAST))

这是保存 FIRRTL AST 的内部数据结构。它是一个根节点为 **Circuit** 的树结构，具有 3 个子节点：**@[file.fir@2.0]**、**ArrayBuffer** 和 **cmd5WrapperHelperDelayBy2**。以下是 `Circuit` 实际被序列化的 Scala 类的定义：<a name="circuit"></a><img src="images/circuit.png" alt="Circuit case class" />

如你所见，它有三个子节点：`info: Info`、`Modules: Seq[DefModule]` 和 `main: String`。它扩展了 `FirrtlNode`，所有 FIRRTL AST 节点都必须这样做。暂时忽略 `def mapXXXX` 函数。

许多 FIRRTL 节点包含一个 `info: Info` 字段，解析器可以在其中插入文件信息，如行号和列号，或插入 `NoInfo` 标记。在这个例子中，**@[file.fir@2.0]** 指的是 FIRRTL 文件的第 2 行，第 0 列。

以下部分将详细介绍所有这些 FIRRTL 节点。

# FIRRTL Node Descriptions

本节描述了 [firrtl/src/main/scala/firrtl/ir/IR.scala](https://github.com/ucb-bar/firrtl/blob/master/src/main/scala/firrtl/ir/IR.scala) 中常见的 FirrtlNodes。

有关未提及组件的更多详细信息，请参阅 [The FIRRTL Specification](https://github.com/ucb-bar/firrtl/blob/master/spec/spec.pdf)。

## Circuit
Circuit 是任何 Firrtl 数据结构的根节点。只有一个 Circuit，并且该 Circuit 包含模块定义列表和顶级模块的名称。

#### FirrtlNode Declaration
```scala 
Circuit(info: Info, modules: Seq[DefModule], main: String)
```

#### Concrete Syntax
```
circuit Adder:
  ... // 模块列表
```
#### In-memory Representation
```scala
Circuit(NoInfo, Seq(...), "Adder")
```

## Module

模块是 Firrtl 中的模块化单位，绝不会直接嵌套（声明模块的实例有其自己的具体语法和 AST 表示）。每个模块都有一个名称、一个端口列表和一个包含其实现的主体。

#### FirrtlNode Declaration
```scala
Module(info: Info, name: String, ports: Seq[Port], body: Stmt) extends DefModule
```

#### Concrete Syntax
```
module Adder:
  ... // 端口列表
  ... // 语句
```
#### In-memory Representation
```scala
Module(NoInfo, "Adder", Seq(...), )
```

## Port
端口定义了模块的一部分 io，具有名称、方向（输入或输出）和类型。

#### FirrtlNode Declaration
```scala
class Port(info: Info, name: String, direction: Direction, tpe: Type)
```
#### Concrete Syntax
```
input x: UInt
```

#### In-memory Representation
```scala
Port(NoInfo, "x", INPUT, UIntType(UnknownWidth))
```

## Statement
语句用于描述模块内的组件及其交互方式。以下是一些常用的语句：

### Block of Statements
一组语句。通常用作模块声明中的 body 字段。

### Wire Declaration
一个包含名称和类型的线声明。它既可以作为源（连接 *from*），也可以作为汇（连接 *to*）。
#### FirrtlNode Declaration
```scala
DefWire(info: Info, name: String, tpe: Type)
```
#### Concrete Syntax
```
wire w: UInt
```
#### In-memory Representation
```scala
DefWire(NoInfo, "w", UIntType(UnknownWidth))
```

### Register Declaration
一个包含名称、类型、时钟信号、复位信号和复位值的寄存器声明。
#### FirrtlNode Declaration
```scala
DefRegister(info: Info, name: String, tpe: Type, clock: Expression, reset: Expression, init: Expression)
```

### Connection
表示从源到汇的方向连接。请注意，它遵循 Chisel 中描述的最后连接语义。

#### FirrtlNode Declaration
```scala
Connect(info: Info, loc: Expression, expr: Expression)
```

### Other Statements
其他语句类型如 `DefMemory`、`DefNode`、`IsInvalid`、`Conditionally` 等在此省略；有关更多详细信息，请参阅 [firrtl/src/main/scala/firrtl/ir/IR.scala](https://github.com/freechipsproject/firrtl/blob/master/src/main/scala/firrtl/ir/IR.scala)。

## Expression
表达式表示对已声明组件的引用或逻辑和算术操作。以下是一些常用的表达式：

### Reference
对已声明组件的引用，例如线、寄存器或端口。它具有名称和类型字段。请注意，它不包含实际声明的指针，而是只包含名称作为字符串。

#### FirrtlNode Declaration
```scala
Reference(name: String, tpe: Type)
```

### DoPrim
一个匿名的原始操作，例如 `Add`、`Sub` 或 `And`、`Or`，或子字选择（`Bits`）。操作的类型由 `op: PrimOp` 字段指示。请注意，所需的参数和常量的数量由 `op` 决定。

#### FirrtlNode Declaration
```scala
DoPrim(op: PrimOp, args: Seq[Expression], consts: Seq[BigInt], tpe: Type)
```

### Other Expressions
其他表达式包括 `SubField`、`SubIndex`、`SubAccess`、`Mux`、`ValidIf` 等在 [firrtl/src/main/scala/firrtl/ir/IR.scala](https://github.com/ucb-bar/firrtl/blob/master/src/main/scala/firrtl/ir/IR.scala) 和 [The FIRRTL Specification](https://github.com/ucb-bar/firrtl/blob/master/spec/spec.pdf) 中有更详细的描述。

# 回到我们的示例

让我们再看看示例中的 FIRRTL AST。希望设计的结构更容易理解了！

In [ ]:
println(stringifyAST(firrtlAST))

That's it for this section! In the next section, we will look at how a FIRRTL transformation walks this AST and modifies it.